# Dashboard to compare various carbon tracker results

## Datasets
- ukb pilot 

## Pipelines
- FreeSurfer: recon-all (1,2,3 stages)
- FastSurfer: FastSurferCNN (volumetric segmentation)

## Monitoring metrics
- experiment duration in hours
- power consumption in kWh (kilo-watt-hours) 

## Carbon trackers
- EIT
- CC
- CT

In [1]:
import sys
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from utils.dashboard_utils import * 

from experiment_impact_tracker.data_interface import DataInterface
from experiment_impact_tracker.data_utils import *
from experiment_impact_tracker.data_utils import (load_data_into_frame,
                                                  load_initial_info,
                                                  zip_data_and_info)

loading region bounding boxes for computing carbon emissions region, this may take a moment...
 454/454... rate=364.28 Hz, eta=0:00:00, total=0:00:01, wall=14:47 EST
Done!


/home/nikhil/projects/green_comp_neuro/experiment-impact-tracker/experiment_impact_tracker/data_interface.py:37: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [2]:
project_dir = '../../'

subject_list_file = f'{project_dir}subject_lists/ukb_pilot_subjects.csv'

tracker_logs_dir = f'{project_dir}tracker_output/'

pipelines = ['FastSurfer/CNN'] # pick from: 'FastSurfer/CNN', 'FastSurfer/recon', 'FreeSurfer'
exp_runs = ['RUN_1']


## Populate paths for different experimental runs
- includes pipeline and run ids

In [3]:
experiment_sets = {}

for pipe in pipelines:
    for run in exp_runs:
        exp_id = f'{pipe}-{run}'
        exp_log_dir = f'{tracker_logs_dir}{pipe}/local_tests/{run}/'
        experiment_sets[exp_id] = exp_log_dir

experiment_sets


{'FastSurfer/CNN-RUN_1': '../../tracker_output/FastSurfer/CNN/local_tests/RUN_1/'}

## Select subjects and trackers

In [4]:
subject_list = ['000']
trackers = ['EIT'] # pick from 'EIT', 'CT'
read_flops = False

for pipe, exp_log_dir in experiment_sets.items():
    print(f'Reading logs for: {pipe}')

    use_cuda = False
    if pipe == 'FastSurfer/CNN':
        use_cuda = True

    for tracker in trackers: 
        print(f'Using carbon tracker: {pipe}')
        
        subject_log_dirs = []
        for sub in subject_list:
            subdir = f'{sub}/{tracker}'
            subject_log_dirs.append(subdir)

        tracker_summary_df_concat, flops_df_concat, power_df_concat = collate_ETI_tracker_data(exp_log_dir, subject_log_dirs, use_cuda, read_flops)
        tracker_summary_df_concat['pipeline'] = pipe
        tracker_summary_df_concat['tracker'] = tracker

        

Reading logs for: FastSurfer/CNN-RUN_1
Using carbon tracker: FastSurfer/CNN-RUN_1


/home/nikhil/projects/green_comp_neuro/experiment-impact-tracker/experiment_impact_tracker/data_utils.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  return json_normalize(json_array, max_level=max_level), json_array


In [5]:
tracker_summary_df_concat

,experiment_name,total_power,total_carbon,PUE,total_duration_papi,total_duration_impact_tracker,pipeline,tracker
0,000/EIT,0.068971,0.027377,1.58,3.115128,3.116415,FastSurfer/CNN-RUN_1,EIT
